## Looking at CAM Budgets

First we set up out environment as before:


In [1]:
# !!! Make sure you are using the cmpi6-2019.10 kernel !!!!

# Add ldcpy root to system path (MODIFY FOR YOUR LDCPY CODE LOCATION)
import sys

sys.path.insert(0, '/glade/u/home/apinard/newldcpy/ldcpy')
import ldcpy

# Display output of plots directly in Notebook
%matplotlib inline
# Automatically reload module if it is editted
%reload_ext autoreload
%autoreload 2

# silence warnings
import warnings

warnings.filterwarnings("ignore")

In [2]:
from dask_jobqueue import PBSCluster

# For Casper
cluster = PBSCluster(
    queue="casper",
    walltime="02:00:00",
    project="NIOW0001",
    memory="40GB",
    resource_spec="select=1:ncpus=4:mem=40GB",
    cores=4,
    processes=1,
)

# for Cheyenne
# cluster = PBSCluster(
#    queue="regular",
#    walltime="02:00:00",
#    project="NIOW0001",
#    memory="109GB",
#    resource_spec="select=1:ncpus=9:mem=109GB",
#    cores=36,
#    processes=9,
# )


# scale as needed
cluster.adapt(minimum_jobs=1, maximum_jobs=30)
cluster

Collections for variables needed for budgets


In [3]:
from dask.distributed import Client

# Connect client to the remote dask workers
client = Client(cluster)
client

Client Scheduler: tcp://10.12.206.54:45270 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/apinard/proxy/{port}/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
fsnt_col = ldcpy.open_datasets(
    "cam-fv",
    ["FSNT"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
flnt_col = ldcpy.open_datasets(
    "cam-fv",
    ["FLNT"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNT.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
precc_col = ldcpy.open_datasets(
    "cam-fv",
    ["PRECC"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECC.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
precl_col = ldcpy.open_datasets(
    "cam-fv",
    ["PRECL"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.PRECL.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
fsns_col = ldcpy.open_datasets(
    "cam-fv",
    ["FSNS"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNS.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
flns_col = ldcpy.open_datasets(
    "cam-fv",
    ["FLNS"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FLNS.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
shflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["SHFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.SHFLX.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
lhflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["LHFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.LHFLX.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
qflx_col = ldcpy.open_datasets(
    "cam-fv",
    ["QFLX"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.QFLX.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05

dataset size in GB 0.05



In [5]:
ldcpy.compare_stats(fsnt_col, "FSNT", ["orig", "mc"], significant_digits=10)

Warning - this data set has a time dimension - examining slice 0 only...


,orig,mc
mean,241.5409968,241.5409742
variance,17747.51086,17747.50376
standard deviation,133.2197841,133.2197574
min value,0,0
max value,432.1576538,432.15625
probability positive,0.9010416667,0.9010416667
number of zeros,5472,5472


,mc
max abs diff,0.05487060547
min abs diff,0
mean abs diff,0.01069879921
mean squared diff,5.11019e-10
root mean squared diff,0.01467808352
normalized root mean squared diff,3.051339187e-05
normalized max pointwise error,0.0001260509364
pearson correlation coefficient,0.9999999951
ks p-value,1
spatial relative error(% > 0.0001),7.125289352


In [6]:
budget_ds = ldcpy.combine_datasets(
    [
        fsns_col,
        fsnt_col,
        shflx_col,
        lhflx_col,
        flnt_col,
        qflx_col,
        precc_col,
        precl_col,
        flns_col,
    ]
)

ldcpy.cam_budgets(
    budget_ds,
    ["orig", "mc", "mg", "ma"],
    significant_digits=6,
)

,orig,mc,mg,ma
restom,0.574528,0.574511,0.574342,0.574487
precip,3.50037e-08,3.50037e-08,3.50038e-08,3.50045e-08
e-p,6.0489,6.0489,6.04892,6.04897
ressurf,1.42711,1.42714,1.42712,1.42667


In [7]:
fsnt = fsnt_col["FSNT"]
flnt = flnt_col["FLNT"]
fsnt_o = fsnt.sel(collection="orig")
fsnt_mc = fsnt.sel(collection="mc")
flnt_o = flnt.sel(collection="orig")
flnt_mc = flnt.sel(collection="mc")

In [8]:
tmp_data_o = fsnt_o - flnt_o
tmp_data_mc = fsnt_mc - flnt_mc

In [9]:
tmp_data_o

<xarray.DataArray (time: 60, lat: 192, lon: 288)>
dask.array<sub, shape=(60, 192, 288), dtype=float32, chunksize=(60, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat         (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon         (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time        (time) object 2006-02-01 00:00:00 ... 2011-01-01 00:00:00
    cell_area   (lat, lon) float64 3.382e-05 3.382e-05 ... 3.382e-05 3.382e-05
    collection  <U4 'orig'
Attributes:
    Sampling_Sequence:  rad_lwsw
    units:              W/m2
    long_name:          Net solar flux at top of model
    cell_methods:       time: mean
    cell_measures:      area: cell_area

In [10]:
import cf_xarray as cf
import xarray as xr

xr.set_options(keep_attrs=True)

fsnt_col = ldcpy.open_datasets(
    "cam-fv",
    ["FSNT"],
    [
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/conservative/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/middle_ground/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
        "/glade/p/cisl/asap/CAM_lossy_test_data_31/aggressive/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc",
    ],
    ["orig", "mc", "mg", "ma"],
)
fsnt = fsnt_col.cf["FSNT"]
fsnt.attrs = fsnt_col.attrs
fsnt_o = fsnt.sel(collection="orig")
fsnt_o.attrs = fsnt.attrs
fsnt_ot = fsnt_o.isel(time=0)
fsnt_ot.attrs = fsnt_o.attrs
# wt_mean = fsnt_ot.cf.weighted("area").mean(["lat","lon"])

wt_mean = fsnt_ot.mean(["lat", "lon"])
wt_mean.compute()

dataset size in GB 0.05



<xarray.DataArray 'FSNT' ()>
array(199.61983, dtype=float32)
Coordinates:
    time        object 2006-02-01 00:00:00
    collection  <U4 'orig'
Attributes:
    Conventions:      CF-1.0
    source:           CAM
    case:             b.e11.BRCP85C5CNBDRD.f09_g16.031
    title:            UNSET
    logname:          mickelso
    host:             ys1023
    Version:          $Name$
    revision_Id:      $Id$
    initial_file:     b.e11.B20TRC5CNBDRD.f09_g16.031.cam.i.2006-01-01-00000.nc
    topography_file:  /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtop...
    history:          Fri May 21 11:59:51 2021: ncks -d time,0,59,1 b.e11.BRC...
    NCO:              netCDF Operators version 4.9.5 (Homepage = http://nco.s...
    cell_measures:    area: cell_area
    data_type:        cam-fv

In [11]:
fsnt.attrs

{'Conventions': 'CF-1.0',
 'source': 'CAM',
 'case': 'b.e11.BRCP85C5CNBDRD.f09_g16.031',
 'title': 'UNSET',
 'logname': 'mickelso',
 'host': 'ys1023',
 'Version': '$Name$',
 'revision_Id': '$Id$',
 'initial_file': 'b.e11.B20TRC5CNBDRD.f09_g16.031.cam.i.2006-01-01-00000.nc',
 'topography_file': '/glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtopo30_0.9x1.25_remap_c051027.nc',
 'history': 'Fri May 21 11:59:51 2021: ncks -d time,0,59,1 b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-208012.nc b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-2007012.n',
 'NCO': 'netCDF Operators version 4.9.5 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)',
 'cell_measures': 'area: cell_area',
 'data_type': 'cam-fv'}

In [12]:
fsnt.attrs

{'Conventions': 'CF-1.0',
 'source': 'CAM',
 'case': 'b.e11.BRCP85C5CNBDRD.f09_g16.031',
 'title': 'UNSET',
 'logname': 'mickelso',
 'host': 'ys1023',
 'Version': '$Name$',
 'revision_Id': '$Id$',
 'initial_file': 'b.e11.B20TRC5CNBDRD.f09_g16.031.cam.i.2006-01-01-00000.nc',
 'topography_file': '/glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtopo30_0.9x1.25_remap_c051027.nc',
 'history': 'Fri May 21 11:59:51 2021: ncks -d time,0,59,1 b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-208012.nc b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-2007012.n',
 'NCO': 'netCDF Operators version 4.9.5 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)',
 'cell_measures': 'area: cell_area',
 'data_type': 'cam-fv'}

In [13]:
fsnt.attrs

{'Conventions': 'CF-1.0',
 'source': 'CAM',
 'case': 'b.e11.BRCP85C5CNBDRD.f09_g16.031',
 'title': 'UNSET',
 'logname': 'mickelso',
 'host': 'ys1023',
 'Version': '$Name$',
 'revision_Id': '$Id$',
 'initial_file': 'b.e11.B20TRC5CNBDRD.f09_g16.031.cam.i.2006-01-01-00000.nc',
 'topography_file': '/glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtopo30_0.9x1.25_remap_c051027.nc',
 'history': 'Fri May 21 11:59:51 2021: ncks -d time,0,59,1 b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-208012.nc b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-2007012.n',
 'NCO': 'netCDF Operators version 4.9.5 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)',
 'cell_measures': 'area: cell_area',
 'data_type': 'cam-fv'}

In [14]:
b = lhflx_col['LHFLX']
if 'time' in b.dims:
    print('yes')

yes


In [15]:
fsnt_col["time"].attrs

{'long_name': 'time', 'bounds': 'time_bnds'}

In [16]:
time_name = "time"
tb_name = fsnt[time_name].attrs["bounds"]
tb_name

'time_bnds'

In [17]:
fsnt_col["time"].attrs

{'long_name': 'time', 'bounds': 'time_bnds'}

In [19]:
# filename = '/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE/atm/proc/tseries/monthly/TS/b.e11.B20TRC5CNBDRD.f09_g16.011.cam.h0.TS.192001-200512.nc'
filename = "/glade/p/cisl/asap/CAM_lossy_test_data_31/orig/b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.FSNT.200601-201012.nc"
ds = xr.open_dataset(filename, decode_times=False)
attrs = ds.time.attrs.copy()
ds = ds.assign_coords(time=ds.time_bnds.mean('nbnd'))
ds.time.attrs = attrs
ds = xr.decode_cf(ds)

In [20]:
ds = xr.open_dataset(filename)
ds.time.dt.year

<xarray.DataArray 'year' (time: 60)>
array([2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006,
       2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007,
       2007, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,
       2008, 2008, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009,
       2009, 2009, 2009, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010,
       2010, 2010, 2010, 2010, 2011])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2011-01-01 00:00:00

In [21]:
months = ds.time.dt.month
years = ds.time.dt.year
years

<xarray.DataArray 'year' (time: 60)>
array([2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006,
       2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007,
       2007, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008,
       2008, 2008, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009, 2009,
       2009, 2009, 2009, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010,
       2010, 2010, 2010, 2010, 2011])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2011-01-01 00:00:00

In [22]:
month_length = fsnt_col.time.dt.days_in_month[0:12]
print(month_length)
wts = month_length.groupby('time.year') / month_length.groupby('time.year').sum()

<xarray.DataArray 'days_in_month' (time: 12)>
array([28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2007-01-01 00:00:00


In [23]:
wts

<xarray.DataArray 'days_in_month' (time: 12)>
array([0.08383234, 0.09281437, 0.08982036, 0.09281437, 0.08982036,
       0.09281437, 0.09281437, 0.08982036, 0.09281437, 0.08982036,
       0.09281437, 1.        ])
Coordinates:
  * time     (time) object 2006-02-01 00:00:00 ... 2007-01-01 00:00:00
    year     (time) int64 2006 2006 2006 2006 2006 ... 2006 2006 2006 2006 2007